In [15]:
import eventDrivenSimulation as eds
import simpy
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import show as sh
import requests
import folium
import haversine
import math
import bibliothek as bib
import random
import simulation as sm
import statistics

## Data Generation

In [2]:
CONFIG = {}

CONFIG['LAT'] = 51.34053598409240
CONFIG['LON'] = 12.381419774766130
CONFIG['RADIUS'] = 500
CONFIG['NUM_UES'] = 400
CONFIG['URL'] = 'https://pqos-api-test.radiolab.dit.htwk-leipzig.de' 

In [3]:
ul_scenario_map = folium.Map(location = [CONFIG['LAT'], CONFIG['LON']], tiles = "cartodbpositron", zoom_start = 15)

ul_query_string = CONFIG['URL'] + '/generate_scenario' + \
                                  '?lat=' + str(CONFIG['LAT']) + \
                                  '&lon=' + str(CONFIG['LON']) + \
                                  '&radius=' + str(CONFIG['RADIUS']) + \
                                  '&num_ues=' + str(CONFIG['NUM_UES']) + \
                                '&cell_type=NGMN3600'

ul_response_data = requests.get(ul_query_string).json()
ue_data = ul_response_data['ue_data']
cell_data=ul_response_data['cell_data']

eds.plot_map(CONFIG,cell_data)

KeyError: 'ue_data'

In [ ]:
df_cell = pd.DataFrame(cell_data)

In [ ]:
sectors=df_cell['pci'].unique()

In [ ]:
bs_dict={}
for i in df_cell.index:
    bs_dict.update({df_cell.loc[i]['pci']:[df_cell.loc[i]['lon'],df_cell.loc[i]['lat']]})

In [ ]:
user_dataframe=pd.DataFrame()
x=np.array([])
for i in df_ue['id']:
    dic=df_ue.loc[i]['cell_info']
    powers={}
    pos=df_ue.loc[i]['pos']
    powers.update({'lat':pos[0]})
    powers.update({'lon':pos[1]})
    for i in dic:
        powers.update({i['pci']:i['dl_rx_power']})
    x=np.append(x,powers)
user_dataframe = pd.DataFrame.from_records(x)

## Preprocessing & CoMP Calculations

Settings for the CoMP Calculations: 
* noise power = -104 dBm

P=vector with receiving powers from all PCIs (N)

**SINR without CoMP**: $\frac{P_{0}}{P_{N}+P_{1:N}}$

**SINR with CoMP (2 coord. PCIs)** : $\frac{(\sqrt{P_{0}}+\sqrt{P_{1}})^2}{P_{N}+P_{2:N}}$

In [ ]:
noise=np.power(10,-104/10) #https://www.sciencedirect.com/topics/engineering/noise-power -> 10MHz bandwidth (50PRBs)
cluster=sectors #all the cells are put in one cluster

In [ ]:
df=user_dataframe.copy()
SINR_list=np.array([])
SINRwithCoMP_list=np.array([])
SINRwithCoMP_phaseshiftList=np.array([])
tp_1_list=np.array([])
tp_2_list=np.array([])
for i in np.arange(0, len(df)):
    P=df.loc[i,:][2:] #goes through file line by line and selects the line
    P=np.power(10, P/10) # calculate the linear values for capacity calculations
    P=P.sort_values(ascending=False) # sort values by their order
    A=np.sqrt(P) #calculate the amplitude
    index=P.index.values #Extract all index values 
    index=index.astype(np.int) #PCIen as integer
    A=np.array(A)
    P=np.array(P)
    SINR= P[0]/(np.sum(P[1:])+noise)
    SINRwithCoMP= np.power(np.sum(A[0:2]),2)/(np.sum(P[2:])+noise)
    #phi=random.gauss(0, math.pi)
    phi=random.gauss(0, math.pi/4)
    SINRwithCoMP_phaseshift=10*np.log10((np.power(A[0]+A[1]*np.cos(phi),2)+np.power(A[1]*np.sin(phi),2))/(np.sum(P[2:])+noise))
    
    SINR_list=np.append(SINR_list,10*np.log10(SINR)) #appends the calculated elements
    SINRwithCoMP_list=np.append(SINRwithCoMP_list,10*np.log10(SINRwithCoMP))
    SINRwithCoMP_phaseshiftList=np.append(SINRwithCoMP_phaseshiftList,SINRwithCoMP_phaseshift)

    tp_1_list=np.append(tp_1_list,index[0])
    tp_2_list=np.append(tp_2_list,index[1])

##### Building the dataframe

In [4]:
df['lat']=user_dataframe['lat']
df['lon']=user_dataframe['lon']
df['SINR [dB]']=SINR_list
df['SINR-CoMP [dB]']=SINRwithCoMP_list
df['SINR-CoMP with phaseshift [dB]']=SINRwithCoMP_phaseshiftList
df['TP1']=tp_1_list.astype(int)
df['TP2']=tp_2_list.astype(int)
df['qos']=np.zeros(len(user_dataframe)) #not needed here -> all users are best effort -> 0
df['id']=np.arange(0,len(user_dataframe))

NameError: name 'user_dataframe' is not defined

Read the existing data

In [4]:
df=pd.read_csv('ue_data_pandas')

#### Define general parameters for the simulation

In [5]:
max_prb=50 #number of prbs defined by the system bandwidth
ue_nr=15 #number of ues per pci
metric=[1,1] #defines the exponents of the metric (standard metric is pf ->[1,1])

#### Sort out PCIs that have less than ue_nr users -> edge of the Scenario

In [6]:
#check if the number of users (positions) for the serving pci is >10
df_filter=df.groupby('TP1')
count=0
for i in df['TP1'].unique():
    k=df_filter.get_group(i)
    if(len(k)<ue_nr):
        cluster=np.delete(cluster,count) #delete cells with less than 10 positions
        df=df[df['TP1']!=i]
    count+=1

#### Define a cluster

In [7]:
sectors= np.array(df['TP1'].unique())
print('pci options for cluster:',sectors)

pci options for cluster: [132 318 774 319 776 320 134 133 775]


In [8]:
#cluster=[134,776,319]
cluster=[320,133]
#cluster=[775,133]
#cluster=[319,775]
#cluster=sectors
#cluster=[133,775,319]

In [9]:
for i in df.index:
    rx_powers=df.iloc[i][3:-7]
    

In [10]:
rx_powers=df.iloc[i][3:-7]

In [11]:
df_filter=df.groupby('TP1')

In [12]:
df2=df_filter.get_group(320)
df_filter2=df2.groupby('TP2')
df2_x=df_filter2.get_group(133)

In [13]:
df3=df_filter.get_group(133)
df_filter3=df3.groupby('TP2')
df3_x=df_filter3.get_group(320)

if following code says "sinr out of range" thats because the sinr is just restricted up to 30 dB

In [14]:
index=np.zeros([len(cluster),ue_nr])
for i in np.arange(0,len(cluster)):
    index[i,:]=np.arange(0,ue_nr)
index=index.astype(int)

size=4000 #size of each packet that arrives from a user [Bit]
env=simpy.Environment()

#ues are initialized 
ue_dict=eds.df_to_ue_lists(df,cluster,6,env) #ue_dict is dict with all users from the dataframe from the defined cluster

# select only ue_nr of users from given index 
ue_per_pci,ue_all=eds.get_user_from_cluster(ue_dict,cluster,ue_nr,index)

Calculate how many bit/ms mean that there is full load 

In [15]:
saver=np.array([])
for i in ue_all:
    saver=np.append(saver,i.tbs)
prbNr_4000_bit_perUE=sum(np.ceil(4000/saver))

In [16]:
print(prbNr_4000_bit_perUE/100,'scheduling events to serve everyone')
print('between', 2*np.floor(prbNr_4000_bit_perUE/100),'ms and',2*np.ceil(prbNr_4000_bit_perUE/100),'ms ~ all UE can be served')

4.35 scheduling events to serve everyone
between 8.0 ms and 10.0 ms ~ all UE can be served


Calculate prb number by relative capacities

In [6]:
a1=0
a2=0
sinr_max=0
for i in ue_all:
    if(i.sinr2>sinr_max):
        sinr_max=i.sinr2
    elif(i.sinr>sinr_max):
        sinr_max=i.sinr
c_max=(np.log2(1+np.exp(sinr_max/10)))
    
for i in ue_all:
    if(i.comp==1):
        sinr=np.exp(i.sinr2/10)
        a2+=c_max*2/(np.log2(1+sinr))
    else:
        sinr=np.exp(i.sinr/10)
        a1+=c_max/(np.log2(1+sinr))
a1/(a1+a2)
a2/(a1+a2)

NameError: name 'ue_all' is not defined

In [20]:
prb_number_mode12=np.array([np.round((eds.calculate_prb_number(ue_all,max_prb)/max_prb)*100),a2/(a1+a2)])

In [35]:
mu_range=np.arange(0,0.3,0.3/10)

In [36]:
mu_range

array([0.  , 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21, 0.24, 0.27])

In [37]:
mu_range=np.arange(0,1,1/40)
mu_range=mu_range[1::]
mu_range=1/mu_range

In [38]:
mu_range

array([40.        , 20.        , 13.33333333, 10.        ,  8.        ,
        6.66666667,  5.71428571,  5.        ,  4.44444444,  4.        ,
        3.63636364,  3.33333333,  3.07692308,  2.85714286,  2.66666667,
        2.5       ,  2.35294118,  2.22222222,  2.10526316,  2.        ,
        1.9047619 ,  1.81818182,  1.73913043,  1.66666667,  1.6       ,
        1.53846154,  1.48148148,  1.42857143,  1.37931034,  1.33333333,
        1.29032258,  1.25      ,  1.21212121,  1.17647059,  1.14285714,
        1.11111111,  1.08108108,  1.05263158,  1.02564103])

In [13]:
prb_number_comp={}
prb_number_comp[320]=19
prb_number_comp[133]=19

#### Different PRB sizes-> Evaluation

In [18]:
metric=[1,1]
sched='pf'
cluster=[133,320]
x_res2=np.array([])
cluster_nr=0
#mu_range=np.arange(0,1,1/40)
#mu_range=mu_range[2::]
#mu_range=1/mu_range
#mu_range=np.arange(0,1,1/20)
#mu_range=mu_range[1::]
mu_range=np.arange(1,10,2)
prb_range=[19]
#prb_range=np.arange(10,40,1)
#prb_range=[17]
fcomp={}
fNocomp={}
mean_rate_Gain_saver={}
mean_rate_Gain={}
noCoMP_rate_saver={}
CoMP_rate_saver={}
CoMP_rate={}
Gain_saver={}
noCoMP_rate={}
load_dict={}
Gain_std_max_min_10_90={}
stat_dic={}
results={}

for prb_number in prb_range:
    Gain_saver={}
    stat_saver={}
    prb_number_comp={}
    load_list=np.array([])
    fairness_Nocomp=np.array([])
    fairness_comp=np.array([])
    result_saver={}
    for i in cluster:
        prb_number_comp[i]=prb_number
    for mu in mu_range:   
        print(mu)
        ue_noCoMP, ue_comp, ue_all, index=sm.run_simulation('deterministic',df,cluster,mu,ue_nr,6,max_prb,prb_number_comp,metric)
        df_res=eds.ue_to_df(ue_all)
        x=np.array((df_res['mR gain']-1)*100)
        x_res2=np.append(x_res2,np.mean(x))
        cluster_nr+=1

        liste=np.array([])
        m1=np.array([])
        m2=np.array([])
        comp=np.array([])
        sinr=np.array([])
        sinr2=np.array([])
        result_2ndsaver={}
        load=0
        for i in ue_all:
            liste=np.append(liste,((i.mR2/i.mR)-1)*100)
            m1=np.append(m1,i.mR)
            m2=np.append(m2,i.mR2)
            load+=np.round(i.queue.level/4000)
            comp=np.append(comp,i.comp)
            sinr=np.append(sinr,i.sinr)
            sinr2=np.append(sinr2,i.sinr2)
        #print('mu=',mu)
        #print('prb-nr=',prb_number)
        #print(load)
        
        load_list=np.append(load_list,load)
        Gain_saver_std_max_min_10_90=np.array([])
        stat=np.array([])
        Gain_saver_std_max_min_10_90=np.append(Gain_saver_std_max_min_10_90,np.mean(liste))
        Gain_saver_std_max_min_10_90=np.append(Gain_saver_std_max_min_10_90,np.max(m2))    
        Gain_saver_std_max_min_10_90=np.append(Gain_saver_std_max_min_10_90,np.min(m2))
        Gain_saver_std_max_min_10_90=np.append(Gain_saver_std_max_min_10_90,np.max(m1))    
        Gain_saver_std_max_min_10_90=np.append(Gain_saver_std_max_min_10_90,np.min(m1))
        Gain_saver_std_max_min_10_90=np.append(Gain_saver_std_max_min_10_90,np.max(liste))    
        Gain_saver_std_max_min_10_90=np.append(Gain_saver_std_max_min_10_90,np.min(liste))
        #Gain_saver_std_max_min_10_90=np.append(Gain_saver_std_max_min_10_90,m2[np.argmax(liste)])
        #Gain_saver_std_max_min_10_90=np.append(Gain_saver_std_max_min_10_90,m2[np.argmin(liste)])
        Gain_saver_std_max_min_10_90=np.append(Gain_saver_std_max_min_10_90,comp[np.argmax(liste)])
        Gain_saver_std_max_min_10_90=np.append(Gain_saver_std_max_min_10_90,comp[np.argmin(liste)])
        stat=np.append(stat,np.std(liste))
        stat=np.append(stat,np.quantile(liste,0.1))
        stat=np.append(stat,np.quantile(liste,0.9))
        stat_saver.update({mu:stat})
        result_2ndsaver.update({"m1":m1})
        result_2ndsaver.update({"m2":m2})
        result_2ndsaver.update({"gain":liste})
        result_2ndsaver.update({"load":load})
        result_2ndsaver.update({"comp":comp})
        result_2ndsaver.update({"sinr":sinr})
        result_2ndsaver.update({"sinr2":sinr2})
        
        result_saver.update({mu:result_2ndsaver})
        
        mean_rate_Gain_saver.update({mu:liste.mean()})   
        noCoMP_rate_saver.update({mu:m1.mean()})
        CoMP_rate_saver.update({mu:m2.mean()})
        Gain_saver.update({mu:Gain_saver_std_max_min_10_90})
        #print(Gain_std_max_min_10_90)
        x=sum(m1)**2/(sum(m1**2)*len(m1))
        x2=sum(m2)**2/(sum(m2**2)*len(m2))
        
        fairness_Nocomp=np.append(fairness_Nocomp,x)
        fairness_comp=np.append(fairness_comp,x2)
   
    l=np.array([])
    l2=np.array([])
    #for i in mu_range:
        #x=speicher.get(i)
        #l=np.append(l,x.mean())
    load_dict.update({prb_number_comp[cluster[0]]:load_list})
    Gain_std_max_min_10_90.update({prb_number_comp[cluster[0]]:Gain_saver})
    mean_rate_Gain.update({prb_number_comp[cluster[0]]:list(mean_rate_Gain_saver.values())})
    noCoMP_rate.update({prb_number_comp[cluster[0]]:list(noCoMP_rate_saver.values())})
    CoMP_rate.update({prb_number_comp[cluster[0]]:list(CoMP_rate_saver.values())})
    fNocomp.update({prb_number_comp[cluster[0]]:fairness_Nocomp})
    fcomp.update({prb_number_comp[cluster[0]]:fairness_comp})
    stat_dic.update({prb_number_comp[cluster[0]]:stat_saver})
    results.update({prb_number_comp[cluster[0]]:result_saver})
    

1
{133: 19, 320: 19}


C:\Users\Caroline Zoll\EDS\eventDrivenSimulation.py:291: RuntimeWarning: divide by zero encountered in double_scalars
  metric=np.append(metric,((i.cp)**e1/(i.mR**e2)))
C:\Users\Caroline Zoll\EDS\eventDrivenSimulation.py:321: RuntimeWarning: divide by zero encountered in double_scalars
  metric=np.append(metric,((cp)**e1/(i.mR2)**e2))


3
{133: 19, 320: 19}
5
{133: 19, 320: 19}
7
{133: 19, 320: 19}
9
{133: 19, 320: 19}


In [52]:
#np.save('PRB_Simulation_133_320_2/Gain_std_max_min_10_90_pf.npy',Gain_std_max_min_10_90)
#np.save('PRB_Simulation_133_320_2/stat_pf.npy',stat_dic)
#np.save('PRB_Simulation_133_320_2/results_pf.npy',results)
#np.save('PRB_Simulation_133_320_2/fainress_CoMP_pf.npy',fcomp)
#np.save('PRB_Simulation_133_320_2/fainress_noCoMP_pf.npy',fNocomp)
#np.save('PRB_Simulation_133_320_2/mean_rate_Gain_pf.npy',mean_rate_Gain)
#np.save('PRB_Simulation_133_320_2/prb_range.npy',prb_range)
#np.save('PRB_Simulation_133_320_2/mu_range.npy',mu_range)

In [ ]:
#np.save('PRB_Simulation_133_320/Gain_std_max_min_10_90_pf.npy',Gain_std_max_min_10_90)
#np.save('PRB_Simulation_133_320/stat_pf.npy',stat_dic)
#np.save('PRB_Simulation_133_320/results_pf.npy',results)

In [3]:
df=pd.read_csv('ue_data_pandas_ext')
ue_nr=15
max_prb=50

In [12]:
#cluster=[133,320]
cluster=[133,320]
x_res2=np.array([])
cluster_nr=0
mu_range=np.arange(1,150,4)
fcomp={}
fNocomp={}
mean_rate_Gain_saver={}
mean_rate_Gain={}
noCoMP_rate_saver={}
CoMP_rate_saver={}
CoMP_rate={}
noCoMP_rate={}
saver=np.array([])
load_saver=np.array([])
ue_dict={}
ut_dict={}

for j in np.arange(1,50):
    fairness_Nocomp=np.array([])
    fairness_comp=np.array([])
    load_list=np.array([])
    ut_list=np.array([])
    for mu in mu_range:
        ue_dict_2={}
        ue_noCoMP, ue_comp, ue_all, index=sm.run_simulation('random',df,cluster,mu,ue_nr,6,max_prb,'calculate',[1,1],'A',2000)
        df_res=eds.ue_to_df(ue_all)
        x=np.array((df_res['mR gain']-1)*100)
        x_res2=np.append(x_res2,np.mean(x))
        cluster_nr+=1

        liste=np.array([])
        m1=np.array([])
        m2=np.array([])
        load=0
        ut=0
        for i in ue_all:
            liste=np.append(liste,((i.mR2/i.mR)-1)*100)
            m1=np.append(m1,i.mR)
            m2=np.append(m2,i.mR2)
            load+=np.round(i.queue.level/4000)
            ut+=i.utilization
            
        ue_dict_2[mu]=ue_all
        load_list=np.append(load_list,load)
        ut_list=np.append(ut_list,ut)
  
        mean_rate_Gain_saver.update({mu:liste.mean()})   
        noCoMP_rate_saver.update({mu:m1.mean()})
        CoMP_rate_saver.update({mu:m2.mean()})
        x=sum(m1)**2/(sum(m1**2)*len(m1))
        x2=sum(m2)**2/(sum(m2**2)*len(m2))
            
        fairness_Nocomp=np.append(fairness_Nocomp,x)
        fairness_comp=np.append(fairness_comp,x2)
    ue_dict[j]=ue_dict_2
        
    k=1
    full_load=0
    load_list2=load_list-30
    k=0
    for i in load_list2:
        if(i<0):
            full_load_ind=k
            print(k)
            break
        k+=1
    load_saver=np.append(load_saver,full_load_ind)
   
    l=np.array([])
    l2=np.array([])
    #for i in mu_range:
        #x=speicher.get(i)
        #l=np.append(l,x.mean())
    
    mean_rate_Gain.update({j:list(mean_rate_Gain_saver.values())})
    noCoMP_rate.update({j:list(noCoMP_rate_saver.values())})
    CoMP_rate.update({j:list(CoMP_rate_saver.values())})
    fNocomp.update({j:fairness_Nocomp})
    fcomp.update({j:fairness_comp})
    saver=np.append(saver,eds.calculate_prb_number_mode2(ue_all,cluster,max_prb,ue_nr)[320])
    ut_dict.update({j:ut_list})

random
{133: 44, 320: 44}
random
{133: 36, 320: 36}
random
{133: 35, 320: 35}
random
{133: 30, 320: 30}
random
{133: 32, 320: 32}
random
{133: 29, 320: 29}
random
{133: 23, 320: 23}
random
{133: 33, 320: 33}
random
{133: 35, 320: 35}
random
{133: 23, 320: 23}
random
{133: 35, 320: 35}
random
{133: 30, 320: 30}
random
{133: 33, 320: 33}
random
{133: 35, 320: 35}
random
{133: 38, 320: 38}
random
{133: 30, 320: 30}
random
{133: 40, 320: 40}
random
{133: 39, 320: 39}
random
{133: 27, 320: 27}
random
{133: 32, 320: 32}
random
{133: 39, 320: 39}
random
{133: 35, 320: 35}
random
{133: 27, 320: 27}
random
{133: 30, 320: 30}
random
{133: 39, 320: 39}
random
{133: 38, 320: 38}
random
{133: 38, 320: 38}
random
{133: 32, 320: 32}
random
{133: 30, 320: 30}
random
{133: 33, 320: 33}
random
{133: 27, 320: 27}
random
{133: 38, 320: 38}
random
{133: 33, 320: 33}
random
{133: 27, 320: 27}
random
{133: 35, 320: 35}
random
{133: 30, 320: 30}
random
{133: 30, 320: 30}
random
{133: 36, 320: 36}
22
random
{1

random
{133: 36, 320: 36}
random
{133: 35, 320: 35}
random
{133: 32, 320: 32}
random
{133: 29, 320: 29}
random
{133: 38, 320: 38}
random
{133: 30, 320: 30}
random
{133: 32, 320: 32}
random
{133: 29, 320: 29}
random
{133: 33, 320: 33}
random
{133: 40, 320: 40}
random
{133: 33, 320: 33}
random
{133: 33, 320: 33}
random
{133: 35, 320: 35}
random
{133: 35, 320: 35}
random
{133: 39, 320: 39}
random
{133: 35, 320: 35}
random
{133: 35, 320: 35}
random
{133: 25, 320: 25}
random
{133: 33, 320: 33}
random
{133: 35, 320: 35}
random
{133: 39, 320: 39}
random
{133: 30, 320: 30}
random
{133: 35, 320: 35}
random
{133: 38, 320: 38}
random
{133: 23, 320: 23}
random
{133: 40, 320: 40}
random
{133: 38, 320: 38}
20
random
{133: 38, 320: 38}
random
{133: 38, 320: 38}
random
{133: 30, 320: 30}
random
{133: 33, 320: 33}
random
{133: 32, 320: 32}
random
{133: 30, 320: 30}
random
{133: 39, 320: 39}
random
{133: 33, 320: 33}
random
{133: 35, 320: 35}
random
{133: 36, 320: 36}
random
{133: 38, 320: 38}
random
{1

random
{133: 30, 320: 30}
random
{133: 39, 320: 39}
random
{133: 33, 320: 33}
random
{133: 38, 320: 38}
random
{133: 29, 320: 29}
random
{133: 35, 320: 35}
random
{133: 27, 320: 27}
random
{133: 35, 320: 35}
random
{133: 30, 320: 30}
random
{133: 36, 320: 36}
random
{133: 30, 320: 30}
random
{133: 36, 320: 36}
random
{133: 35, 320: 35}
random
{133: 35, 320: 35}
random
{133: 30, 320: 30}
random
{133: 35, 320: 35}
22
random
{133: 25, 320: 25}
random
{133: 41, 320: 41}
random
{133: 30, 320: 30}
random
{133: 38, 320: 38}
random
{133: 32, 320: 32}
random
{133: 40, 320: 40}
random
{133: 36, 320: 36}
random
{133: 33, 320: 33}
random
{133: 27, 320: 27}
random
{133: 30, 320: 30}
random
{133: 27, 320: 27}
random
{133: 33, 320: 33}
random
{133: 33, 320: 33}
random
{133: 33, 320: 33}
random
{133: 36, 320: 36}
random
{133: 38, 320: 38}
random
{133: 30, 320: 30}
random
{133: 25, 320: 25}
random
{133: 32, 320: 32}
random
{133: 36, 320: 36}
random
{133: 32, 320: 32}
random
{133: 32, 320: 32}
random
{1

random
{133: 29, 320: 29}
random
{133: 39, 320: 39}
random
{133: 33, 320: 33}
random
{133: 36, 320: 36}
random
{133: 29, 320: 29}
23
random
{133: 33, 320: 33}
random
{133: 32, 320: 32}
random
{133: 36, 320: 36}
random
{133: 35, 320: 35}
random
{133: 32, 320: 32}
random
{133: 30, 320: 30}
random
{133: 33, 320: 33}
random
{133: 36, 320: 36}
random
{133: 38, 320: 38}
random
{133: 35, 320: 35}
random
{133: 35, 320: 35}
random
{133: 30, 320: 30}
random
{133: 30, 320: 30}
random
{133: 29, 320: 29}
random
{133: 36, 320: 36}
random
{133: 30, 320: 30}
random
{133: 29, 320: 29}
random
{133: 35, 320: 35}
random
{133: 30, 320: 30}
random
{133: 29, 320: 29}
random
{133: 39, 320: 39}
random
{133: 35, 320: 35}
random
{133: 36, 320: 36}
random
{133: 33, 320: 33}
random
{133: 38, 320: 38}
random
{133: 32, 320: 32}
random
{133: 33, 320: 33}
random
{133: 33, 320: 33}
random
{133: 35, 320: 35}
random
{133: 36, 320: 36}
random
{133: 36, 320: 36}
random
{133: 30, 320: 30}
random
{133: 33, 320: 33}
random
{1

random
{133: 39, 320: 39}
random
{133: 35, 320: 35}
random
{133: 38, 320: 38}
random
{133: 35, 320: 35}
random
{133: 36, 320: 36}
random
{133: 33, 320: 33}
random
{133: 33, 320: 33}
random
{133: 30, 320: 30}
random
{133: 38, 320: 38}
random
{133: 25, 320: 25}
random
{133: 40, 320: 40}
random
{133: 17, 320: 17}
random
{133: 35, 320: 35}
random
{133: 32, 320: 32}
random
{133: 30, 320: 30}
random
{133: 39, 320: 39}
random
{133: 36, 320: 36}
random
{133: 36, 320: 36}
random
{133: 33, 320: 33}
random
{133: 39, 320: 39}
random
{133: 32, 320: 32}
random
{133: 36, 320: 36}
random
{133: 36, 320: 36}
random
{133: 36, 320: 36}
random
{133: 30, 320: 30}
random
{133: 36, 320: 36}
random
{133: 32, 320: 32}
random
{133: 32, 320: 32}
random
{133: 36, 320: 36}
random
{133: 30, 320: 30}
random
{133: 36, 320: 36}
random
{133: 33, 320: 33}
22
random
{133: 39, 320: 39}
random
{133: 35, 320: 35}
random
{133: 32, 320: 32}
random
{133: 39, 320: 39}
random
{133: 38, 320: 38}
random
{133: 33, 320: 33}
random
{1

random
{133: 40, 320: 40}
random
{133: 17, 320: 17}
random
{133: 36, 320: 36}
random
{133: 38, 320: 38}
random
{133: 29, 320: 29}
random
{133: 33, 320: 33}
random
{133: 39, 320: 39}
random
{133: 35, 320: 35}
random
{133: 27, 320: 27}
random
{133: 35, 320: 35}
random
{133: 32, 320: 32}
random
{133: 39, 320: 39}
random
{133: 35, 320: 35}
random
{133: 39, 320: 39}
random
{133: 35, 320: 35}
random
{133: 32, 320: 32}
random
{133: 39, 320: 39}
random
{133: 33, 320: 33}
random
{133: 36, 320: 36}
random
{133: 35, 320: 35}
random
{133: 39, 320: 39}
20
random
{133: 32, 320: 32}
random
{133: 33, 320: 33}
random
{133: 38, 320: 38}
random
{133: 35, 320: 35}
random
{133: 32, 320: 32}
random
{133: 33, 320: 33}
random
{133: 35, 320: 35}
random
{133: 41, 320: 41}
random
{133: 32, 320: 32}
random
{133: 35, 320: 35}
random
{133: 33, 320: 33}
random
{133: 33, 320: 33}
random
{133: 32, 320: 32}
random
{133: 35, 320: 35}
random
{133: 42, 320: 42}
random
{133: 36, 320: 36}
random
{133: 32, 320: 32}
random
{1

In [13]:
ut_dict

{1: array([2970., 2964., 2965., 2954., 2868., 2604., 2522., 2793., 2419.,
        2154., 2387., 1551., 2108.,  756., 1678., 1338., 1129.,  938.,
        1437., 1108., 1042.,  834.,  525., 1006.,  656.,  575.,  252.,
         222.,  327.,  293.,  322.,  109.,  118.,  148.,  116.,  316.,
         186.,  105.]),
 2: array([2969., 2963., 2964., 2922., 2952., 2948., 2657., 2815., 2411.,
        2139., 2465., 2107., 2149., 1674., 1544., 1564., 1804., 1183.,
        1479., 1401., 1374.,  317.,  602.,  292.,  323.,  786.,  549.,
         164.,  273.,  370.,  341.,  534.,  196.,  300.,  273.,  173.,
         175.,  272.]),
 3: array([2969., 2966., 2961., 2949., 2951., 2692., 2709., 2628., 2153.,
        2500., 1815., 2032., 1463., 1922., 1429., 1357., 1159., 1169.,
        1449., 1267.,  757.,  556.,  547.,  500.,  360.,  639.,  295.,
         229.,  329.,  573.,  344.,  275.,  342.,  102.,  167.,  204.,
         208.,   87.]),
 4: array([2969., 2965., 2956., 2949., 2916., 2855., 2305., 2090., 

In [25]:
result={}
result['noCoMP_rate']=noCoMP_rate
result['CoMP_rate']=CoMP_rate
result['fcomp']=fcomp
result['fNocomp']=fNocomp
result['mean_rate_Gain']=mean_rate_Gain
result['load_saver']=load_saver
result['mu_range']=mu_range
result['prb res']=saver
result['prb res']=saver
#np.save('50rounds/result-5UEs-SA.npy',result)

In [23]:
load_saver

array([3., 3., 3., 2., 2., 2., 3., 3., 2., 3., 3., 3., 2., 2., 2., 3., 3.,
       2., 2., 3., 3., 2., 4., 3., 2., 3., 3., 3., 3., 3., 2., 3., 4., 2.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3., 4., 1.])

In [24]:
np.mean(load_saver)

2.7346938775510203